# Experiments on VQC
### Main experiment (Figure 4)

In [ ]:
from qiskit import IBMQ
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-research')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np

from qiskit.test.mock import FakeJakarta

from qiskit.utils import QuantumInstance
from qiskit.providers.aer.backends import AerSimulator
from qiskit.aqua import aqua_globals
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit_machine_learning.circuit.library import RawFeatureVector

from qclib.machine_learning.baa_feature_vector import BaaFeatureVector
from qclib.machine_learning.vqc import VQC
from qclib.machine_learning.datasets import digits

seed = 42

In [ ]:
# Dataset load.

sample_total, training_input, test_input, class_labels = digits.load(classes=[0, 1],
                                                                      training_size=40,
                                                                      test_size=10,
                                                                      random_seed=seed)
feature_dim = len(training_input[class_labels[0]][0])

In [ ]:
# Parameterized circuit model

n_qubits = int(np.ceil(np.log2(feature_dim)))
var_form = TwoLocal(n_qubits, ['ry', 'rz'], 'cz', reps=1)
var_form.decompose().draw()

In [ ]:
def backend_info(backend):
    backend_name = backend.name()
    backend_qubits = backend.configuration().n_qubits
    print('Experiments using {0} backend, with {1} qubits available.'.format(backend_name, backend_qubits))

def ideal_instance():
    ideal = AerSimulator()
    return QuantumInstance(ideal, shots=1024, seed_simulator=seed, seed_transpiler=seed)

def noisy_instance():
    device = FakeJakarta()
    noisy = AerSimulator.from_backend(device)
    return QuantumInstance(noisy, shots=1024, seed_simulator=seed, seed_transpiler=seed)

def experiment(l, reps, quantum_instance, vqc):
    acc = []
    for i in range(reps):
        print(f'l\t{l}\trun\t{i}\t', end='')

        result = vqc.run(quantum_instance)
        acc.append(result["testing_accuracy"])

        print(f'acc\t{result["testing_accuracy"]}')

    print(f'l\t{l}\t--\t--\tavg\t{sum(acc) / len(acc)}\tstd\t{np.std(acc)}')

In [ ]:
# VQC experiments

# Config.
run_qiskit = True
run_baa = True
run_ideal = True
run_noisy = True

reps = 1

baa_strategy = 'brute_force'
baa_low_rank = True
baa_l_max = [i/10 for i in range(11)]
# End config.

aqua_globals.random_seed = seed
algorithm_globals.random_seed = seed

optimizer = SPSA(maxiter=200)
quantum_instances = []
training_size = sum([ len(val) for key, val in training_input.items() ] )
batch_size = int(training_size * 0.1)

if run_ideal:
    quantum_instances.append(ideal_instance())
if run_noisy:
    quantum_instances.append(noisy_instance())

for quantum_instance in quantum_instances:
    backend_info(quantum_instance.backend)

    # Experiment using qiskit initialize function as feature map.
    if run_qiskit:
        print('qiskit')
        qiskit_feature_map = RawFeatureVector(feature_dimension=feature_dim)
        vqc = VQC(optimizer, qiskit_feature_map, var_form, training_input, test_input, None, minibatch_size=batch_size)
        experiment(0.0, reps, quantum_instance, vqc)

    # Experiment using BAA initialize function as feature map.
    if run_baa:
        print('baa')
        for l in baa_l_max:
            baa_feature_map = BaaFeatureVector(feature_dimension=feature_dim, 
                                               strategy=baa_strategy,
                                               max_fidelity_loss=l,
                                               use_low_rank=baa_low_rank)
            
            vqc = VQC(optimizer, baa_feature_map, var_form, training_input, test_input, None, minibatch_size=batch_size)
            experiment(l, reps, quantum_instance, vqc)